In [ ]:
# Imports, always include this at the beginning

# No science without NumPy
import numpy as np

# Matplotlib for plotting
import matplotlib.pyplot as plt
%matplotlib notebook

# for pretty printing the model
from IPython.display import display, Markdown

# Of course we want Glotaran
import glotaran as gta

In [ ]:
# Load the data
dataset = gta.io.WavelengthExplicitFile("HIBA1172DY_2.ascii").read("dataset1")
data = dataset.get()
time_axis = dataset.get_time_axis()
spectral_axis = dataset.get_spectral_axis()


In [ ]:
# Plot the Data

plt.figure(figsize=(20, 20))

# Plot some time traces
plt.subplot(4, 4, 1)
wl = [700, 690]
plt.title(f'Traces at {wl[0]}, {wl[1]}')
for w in wl:
    i = (np.abs(spectral_axis-w)).argmin()
    plt.plot(time_axis, data[i, :])

# Full Data
plt.subplot(4, 4, 2)
plt.title('Data')
plt.pcolormesh(time_axis, spectral_axis, data)

# Plot the spectrum at time=-83
time_zero = -83
time_zero_idx = (np.abs(time_axis-time_zero)).argmin()
plt.subplot(4, 4, 3)
plt.title('Spectrum at t=-83ps')
plt.plot(spectral_axis, data[:, time_zero_idx])


plt.figure(figsize=(20, 20))

lsvd, svals, rsvd = dataset.svd()

# Plot left singular vectors (LSV, times, first 4)
plt.subplot(4, 4, 1)
plt.title('LSV Data')
for i in range(4):
    plt.plot(time_axis, lsvd[:, i])

# Plot singular values (SV)
plt.subplot(4, 4, 2)
plt.title('SVals Data')
plt.plot(range(max(10, min(len(time_axis), len(spectral_axis)))), svals, 'ro')
plt.yscale('log')

# Plot right singular vectors (RSV, wavelengths, first 4)
plt.subplot(4, 4, 3)
plt.title('RSV Data')
for i in range(4):
    plt.plot(spectral_axis, rsvd[:, i])

In [ ]:
# Then we can create our model

model_spec = '''
type: kinetic
parameters: 
  - initial_concentration:
    - [1, {vary: false}]
    - [0, {vary: false}]
    - [0, {vary: false}]
    - [0, {vary: false}]
  - irf:
    - ["center", 0.275649637 ]
    - ["width", 8.73516053E-02]
    - [13200.0, "backsweep", {vary: false}]
    - ["center_dispersion", [0.462641180, 0.129131436]]
  - k:
    - 3.00191021 
    - 0.398853779
    - 8.44753608E-02
    - 3.31102014E-02
  - osc:
    - [41196.9023, 172.540680]
    - [12.9041157, 256.013367]
    - [222.555908, 105.456551]
    - [88.8726578, 45.5685043] 
                              
compartments: [s1, s2, s3, s4, doas1, doas2, doas3, doas4]

megacomplexes:
    - label: mc1
      k_matrices: [km1]
      oscillations: [o1, o2, o3, o4]
      
k_matrices:
  - label: "km1"
    matrix: {
      '("s2","s1")': k.1,
      '("s3","s2")': k.2,
      '("s4","s3")': k.3,
      '("s4","s4")': k.4
    }

oscilations:
  - label: o1
    comp: doas1
    frequency: osc.1[0]
    decay: osc.1[1]
  - label: o2
    comp: doas2
    frequency: osc.2[0]
    decay: osc.2[1]
  - label: o3
    comp: doas3
    frequency: osc.3[0]
    decay: osc.3[1]
  - label: o4
    comp: doas4
    frequency: osc.4[0]
    decay: osc.4[1]    
    
irf:
  - label: irf
    type: gaussian
    center: irf.center
    width: irf.width
    center_dispersion: irf.center_dispersion

initial_concentration: #equal to the total number of compartments
  - label: inputD1
    parameter: [initial_concentration.1, initial_concentration.2, initial_concentration.3, initial_concentration.4] 
    
datasets:
  - label: dataset1
    type: spectral
    initial_concentration: inputD1
    megacomplexes: [mc1]
    path: ''
    irf: irf
'''

model = gta.parse(model_spec)
# print(model)
#model.set_dataset("dataset1", dataset)
display(Markdown(str(model)))

In [ ]:
result = model.fit()
display(Markdown(str(result)))

In [ ]:
# Plot the fitted data

fitted_dataset = result.fitted_data('dataset1')
data = fitted_dataset.get()
# Plot the Data

plt.figure(figsize=(20, 20))

# Plot left singular vectors (LSV, times, first 4)
plt.subplot(4, 4, 1)
wl = [700, 690]
plt.title(f'Traces at {wl[0]}, {wl[1]}')
for w in wl:
    i = (np.abs(spectral_axis-w)).argmin()
    plt.plot(time_axis, data[i, :])

# Full Data
plt.subplot(4, 4, 2)
plt.title('Data')
plt.pcolormesh(time_axis, spectral_axis, data)

time_zero = -83
time_zero_idx = (np.abs(time_axis-time_zero)).argmin()
plt.subplot(4, 4, 3)
plt.title('Spectrum at t=-83ps')
plt.plot(spectral_axis, data[:, time_zero_idx])


plt.figure(figsize=(20, 20))

residual_svd = result.final_residual_svd()
lsvd = residual_svd[2] 
svals = residual_svd[1]
rsvd = residual_svd[0] 

# Plot left singular vectors (LSV, times, first 4)
plt.subplot(4, 4, 1)
plt.title('LSV Residual')
for i in range(4):
    plt.plot(time_axis, lsvd[:, i])

# Plot singular values (SV)
plt.subplot(4, 4, 2)
plt.title('SVals Residual')
plt.plot(range(max(10, min(len(time_axis), len(spectral_axis)))), svals, 'ro')
plt.yscale('log')

# Plot right singular vectors (RSV, wavelengths, first 4)
plt.subplot(4, 4, 3)
plt.title('RSV Residual')
for i in range(4):
    plt.plot(spectral_axis, rsvd[:, i])